In [ ]:
# Import Library Dependencies
from sqlalchemy import create_engine, inspect, exc
from sqlalchemy import Column, ForeignKey
from sqlalchemy import Float, Integer, String
from sqlalchemy import Boolean, DateTime, Numeric
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

from datetime import datetime
from config import user, password

In [ ]:
Base = declarative_base()

In [ ]:
# Create Student Class -- Main

class Student(Base):
    __tablename__ = 'students'
    student_id = Column(Integer, primary_key=True, nullable=False)
    student_fname = Column(String(100), nullable=False)
#    student_mname = Column(String(100))
    student_lname = Column(String(100), nullable=False)
    student_age = Column(Integer)
    student_residence_id = Column(Integer, ForeignKey('cities.city_id'))
    updated_on = Column(DateTime, onupdate=datetime.now())
    updated_by = Column(String(255))
    
    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'{self.student_id}, '
                f'{self.student_fname}, '
#                f'{self.student_mname}, '
                f'{self.student_lname}, '
                f'{self.student_age}, '
                f'{self.student_residence_id}, '
                f'{self.updated_on}, '
                f'{self.updated_by})')

In [ ]:
# Student Record Model - Database Table Configuration

class Student_Record(Base):
    __tablename__ = 'student_records'
    record_index = Column(Integer, primary_key=True, autoincrement=True)
    student_id = Column(Integer, ForeignKey('students.student_id'))
    gifted_student = Column(Boolean, default=False)
    disability_student = Column(Boolean, default=False)
    esl_student = Column(Boolean, default=False)
#    minority_student = Column(Boolean, default=False)
#    student_school_district = Column(String(10), nullable=False)
    student_school_id = Column(Integer, ForeignKey('schools.school_id'))
    student_entry_date = Column(DateTime, nullable=False)
    student_program_id = Column(String(5))#, ForeignKey('school_programs.program_id'))
    student_grade_level = Column(Integer, nullable=False)
    student_leave_date = Column(DateTime, nullable=False)
    student_exit_reason = Column(Integer, nullable=False)    
    
    student = relationship('Student', backref=backref('student_records'))
    school = relationship('School', backref=backref('schools'))
    #program = relationship('School_Program', backref=backref('school_programs'))
    
    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'{self.record_index}, '
                f'{self.student_id}, '
                f'{self.gifted_student}, '
                f'{self.disability_student}, '
                f'{self.esl_student}, '
#                f'{self.minority_student}, '
#                f'{self.student_school_district}, '
                f'{self.student_school_id}, '
                f'{self.student_entry_date}, '
                f'{self.student_grade_level}, '
                f'{self.student_leave_date}, '
                f'{self.student_exit_reason})')

In [ ]:
class School(Base):
    __tablename__ =  'schools'
    school_id = Column(Integer, primary_key=True, nullable=False)
    school_name = Column(String(255), nullable=False)
    school_address = Column(String(255), nullable=True)
#    school_city_id = Column(Integer, ForeignKey('cities.city_id'))
    school_district = Column(String(10), nullable=False)
    
    student_record = relationship('Student_Record', backref=backref('schools'))
    school_program = relationship('School_Program', backref=backref('school_programs'))
    
    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'{self.school_id}, '
                f'{self.school_name}, '
                f'{self.school_address}, '
                f'{self.school_city}, '
                f'{self.district})')

In [ ]:
class School_Program(Base):
    __tablename__ = 'school_programs'
    #program_index = Column(Integer, primary_key=True)
    program_id = Column(String(5), primary_key=True)
    program_name = Column(String(255), nullable=False)
#    program_type = Column(String(10))
    program_description = Column(String(255))
    program_start_date = Column(DateTime, nullable=True)
    program_end_date = Column(DateTime, nullable=True)
#    program_school_id = Column(Integer, ForeignKey('schools.school_id'))
    
    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'{self.program_id}, '
                f'{self.program_name}, '
#                f'{self.program_type}, '
                f'{self.program_description}, '
                f'{self.program_start_date}, '
                f'{self.program_end_date}, '
                f'{self.program_school_id})')

In [ ]:
class City(Base):
    __tablename__ = 'cities'
    city_id =  Column(Integer, primary_key=True, nullable=False)
    city_code = Column(String(5))
    city_name = Column(String(255), nullable=False)
    city_median_income = Column(Integer)
    city_population = Column(Integer)
    city_crime_rate = Column(Float)
    
    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'{self.city_id}, '
                f'{self.city_code}, '
                f'{self.city_name}, '
                f'{self.city_median_income}, '
                f'{self.city_population}, '
                f'{self.city_crime_rate})')

In [ ]:
# MySQL Database Connection String
MySQL_DB_Connection = f'{user}:{password}@localhost/student_mobility'

In [ ]:
print(MySQL_DB_Connection)

In [ ]:
# Establish Connection to Database
try:
    # Try to Connect to Existing StudentMobilityDB
    engine = create_engine(f'mysql://{MySQL_DB_Connection}', echo=False)
    connection = engine.connect()
    print(f'\n<Connected> MySQL Database: student_mobility')

except exc.OperationalError:
    # Existing Database Not Exist: Create New MySQL Database: Name='StudentMobilityDB'
    MySQL_Base_Connection = f'{user}:{password}@localhost'
    engine = create_engine(f'mysql://{MySQL_Base_Connection}', echo=False)
    connection = engine.connect()
    engine.execute('CREATE DATABASE IF NOT EXISTS student_mobility;')
    engine = create_engine(f'mysql://{MySQL_DB_Connection}', echo=False)
    print(f'\n<Created> New MySQL Database: student_mobility [Connection: Established]')

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Preview Table Fields - "cities"
cities = [city['name'] for city in inspector.get_columns('cities')]
cities

In [ ]:
# Preview Table Fields - "schools"
schools = [school['name'] for school in inspector.get_columns('schools')]
schools

In [ ]:
# Preview Table Fields - "students"
students = [student['name'] for student in inspector.get_columns('students')]
students

In [ ]:
# Preview Table Fields - "student_records"
records = [rec['name'] for rec in inspector.get_columns('student_records')]
records

In [ ]:
print('All done :)')